In [1]:
import m_nn, d_mg5_data
import torch
import pennylane as qml
from pennylane import numpy as np

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [2]:
gnn_idx_qubits = 2
dataset = d_mg5_data.FatJetEvents(channel="VzToZhToVevebb", cut_pt=(800, 1000), subjet_radius=0.1, num_pt_ptcs=2**gnn_idx_qubits)
events  = dataset.generate_uniform_pt_events(bin=10, num_bin_data=10)

DataLog: Now loading hdf5 file VzToZhToVevebb|c800_1000_r0.1.hdf5
DataLog: Successfully loading hdf5 file VzToZhToVevebb|c800_1000_r0.1.hdf5
DataLog: Generate uniform Pt events (1/10) | number of bin events = 10/11075
DataLog: Generate uniform Pt events (2/10) | number of bin events = 10/11328
DataLog: Generate uniform Pt events (3/10) | number of bin events = 10/11761
DataLog: Generate uniform Pt events (4/10) | number of bin events = 10/11922
DataLog: Generate uniform Pt events (5/10) | number of bin events = 10/11775
DataLog: Generate uniform Pt events (6/10) | number of bin events = 10/11568
DataLog: Generate uniform Pt events (7/10) | number of bin events = 10/11036
DataLog: Generate uniform Pt events (8/10) | number of bin events = 10/10258
DataLog: Generate uniform Pt events (9/10) | number of bin events = 10/8917
DataLog: Generate uniform Pt events (10/10) | number of bin events = 10/7399


In [3]:
# def ctrl_enc_operator(_input, control, control_values):
#     ctrl = qml.ctrl(qml.AngleEmbedding, control=control, control_values=control_values)
#     ctrl(features=_input, wires=range(gnn_idx_qubits, gnn_idx_qubits+3), rotation="Y")
# model = m_nn.QuantumDisorderedFCGraph(num_idx_qubits=gnn_idx_qubits, num_nn_qubits=3, num_layers=1, num_reupload=0, ctrl_enc_operator=ctrl_enc_operator, rnd_shuffle_inputs=True)

def ctrl_enc_operator(_input, control, control_values):
    ctrl_H = qml.ctrl(qml.Hadamard, control=control, control_values=control_values)
    ctrl_H(wires=gnn_idx_qubits)
    ctrl_R = qml.ctrl(qml.Rot, control=control, control_values=control_values)
    ctrl_R(theta=_input[0], phi=_input[1], omega=_input[2], wires=gnn_idx_qubits)
model = m_nn.QuantumDisorderedFCGraph(num_idx_qubits=gnn_idx_qubits, num_nn_qubits=2, num_layers=1, num_reupload=0, ctrl_enc_operator=ctrl_enc_operator)

fatjet_radius = 0.8
f1 = np.arctan(events["fast_pt"] / events["fatjet_pt"])
f2 = events["fast_delta_eta"] / fatjet_radius * (np.pi/2)
f3 = events["fast_delta_phi"] / fatjet_radius * (np.pi/2)

idx  = torch.randint(0, len(events), (1,)).item()
data = torch.tensor([f1[idx], f2[idx], f3[idx]]).mT
for _ in range(10):
    data = data[torch.randperm(2**gnn_idx_qubits)]
    print(model(data.reshape(-1)))


tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
tensor([-2.7588,  0.6569], dtype=torch.float64, grad_fn=<SumBackward1>)
